# Data processing

In this notebook we will combine the different data sources collected for the project.

We had to include external data sources to out first options due to the original dataset being almost only composed of AI-generated text

## Imports

In [1]:
# Set root path
import sys

sys.path.append("..")

import os
import re
import logging
from typing import List

logger = logging.getLogger(__name__)

import polars as pl
from cfg import CFG

from data import *

In [2]:
sources = load_and_merge_sources()

In [3]:
# Plot distribution of each column
for col in sources.columns:
    print(sources[col].value_counts())

shape: (228_699, 2)
┌─────────────────────────────────┬───────┐
│ text                            ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ I believe that homework is an … ┆ 4     │
│ Driverless cars are coming. Dr… ┆ 5     │
│ Ancient Hawaiian Astronomers a… ┆ 1     │
│ David Hume: The Ideology of Se… ┆ 1     │
│ History to the Eighteenth Cent… ┆ 1     │
│ …                               ┆ …     │
│ Dear Senator,                   ┆ 5     │
│                                 ┆       │
│ I believe that …                ┆       │
│ While driverless cars present … ┆ 2     │
│ *Your reasons for wanting to s… ┆ 1     │
│ The Watergate Scandal’s Source… ┆ 1     │
│ Spotlight on Ford Motor Compan… ┆ 1     │
└─────────────────────────────────┴───────┘
shape: (2, 2)
┌───────────┬────────┐
│ generated ┆ count  │
│ ---       ┆ ---    │
│ i8        ┆ u32    │
╞═══════════╪════════╡
│ 1         ┆ 2

In [4]:
sources

text,generated,source
str,i8,str
"""There are a variety of opinion…",1,"""machine-dev.csv"""
"""The university education is no…",1,"""machine-dev.csv"""
"""I believe that the university …",1,"""machine-dev.csv"""
"""University education is a topi…",1,"""machine-dev.csv"""
"""The purpose of university educ…",1,"""machine-dev.csv"""
…,…,…
"""Essay 1Although military analo…",0,"""essay_forum_real.csv"""
"""Through my experiences in life…",0,"""essay_forum_real.csv"""
"""Since I am in Singapore, I hav…",0,"""essay_forum_real.csv"""
